Draft of a crypto punk model
* Add your infura key to env under KEY
* web3.py only works w python 3.7 for some reason
* one hot encoding of attributes(eg: hair color etc) and punk types (ape, alien, etc)
* gets the "punkBought" from the punks contract for price data


In [331]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import glob
import os
from web3 import Web3

In [521]:
# get attribute data
data_filenames = glob.glob("./data/*.csv")

df = pd.DataFrame()
for f in data_filenames:
    df = df.append(pd.read_csv(f))

df = df.set_index('id')
df.columns = ['type', 'attr_count', 'attrs']


def clean(attr_str):
    if (attr_str == ' '): 
        return ['None']
    else:
        attrs = attr_str.split(" / ")
        return list(map(lambda x: x.replace(" ",""), attrs))

df['attrs'] = df['attrs'].apply(clean)#https://stackoverflow.com/questions/29034928/pandas-convert-a-column-of-list-to-dummies/29036042
attr_dummies = pd.get_dummies(df['attrs'].apply(pd.Series).stack()).sum(level=0)
type_dummies = pd.get_dummies(df['type'])
type_dummies = type_dummies.rename(str.strip, axis='columns')
del df['attrs']
del df['type']

df = df.join(type_dummies).join(attr_dummies)

In [507]:
def hexDaysAgo(w3, days):
    blocks_per_day = round(86400 / 14)
    target = w3.eth.block_number - days * blocks_per_day
    return hex(target)

In [528]:
INFURA_KEY = os.environ.get('KEY')
# print(INFURA_KEY)

None


In [515]:
# get events

QUERY_DAYS = 30
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/a92bf60569df4e02b242dbd8357fffac'))
abi_text = """[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"punksOfferedForSale","outputs":[{"name":"isForSale","type":"bool"},{"name":"punkIndex","type":"uint256"},{"name":"seller","type":"address"},{"name":"minValue","type":"uint256"},{"name":"onlySellTo","type":"address"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"punkIndex","type":"uint256"}],"name":"enterBidForPunk","outputs":[],"payable":true,"type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"punkIndex","type":"uint256"},{"name":"minPrice","type":"uint256"}],"name":"acceptBidForPunk","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"addresses","type":"address[]"},{"name":"indices","type":"uint256[]"}],"name":"setInitialOwners","outputs":[],"payable":false,"type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"imageHash","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"nextPunkIndexToAssign","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"punkIndexToAddress","outputs":[{"name":"","type":"address"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"standard","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"punkBids","outputs":[{"name":"hasBid","type":"bool"},{"name":"punkIndex","type":"uint256"},{"name":"bidder","type":"address"},{"name":"value","type":"uint256"}],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[],"name":"allInitialOwnersAssigned","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"allPunksAssigned","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"punkIndex","type":"uint256"}],"name":"buyPunk","outputs":[],"payable":true,"type":"function"},{"constant":false,"inputs":[{"name":"to","type":"address"},{"name":"punkIndex","type":"uint256"}],"name":"transferPunk","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"punkIndex","type":"uint256"}],"name":"withdrawBidForPunk","outputs":[],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"to","type":"address"},{"name":"punkIndex","type":"uint256"}],"name":"setInitialOwner","outputs":[],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"punkIndex","type":"uint256"},{"name":"minSalePriceInWei","type":"uint256"},{"name":"toAddress","type":"address"}],"name":"offerPunkForSaleToAddress","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"punksRemainingToAssign","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"punkIndex","type":"uint256"},{"name":"minSalePriceInWei","type":"uint256"}],"name":"offerPunkForSale","outputs":[],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"punkIndex","type":"uint256"}],"name":"getPunk","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"pendingWithdrawals","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"punkIndex","type":"uint256"}],"name":"punkNoLongerForSale","outputs":[],"payable":false,"type":"function"},{"inputs":[],"payable":true,"type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"punkIndex","type":"uint256"}],"name":"Assign","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"punkIndex","type":"uint256"}],"name":"PunkTransfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"punkIndex","type":"uint256"},{"indexed":false,"name":"minValue","type":"uint256"},{"indexed":true,"name":"toAddress","type":"address"}],"name":"PunkOffered","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"punkIndex","type":"uint256"},{"indexed":false,"name":"value","type":"uint256"},{"indexed":true,"name":"fromAddress","type":"address"}],"name":"PunkBidEntered","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"punkIndex","type":"uint256"},{"indexed":false,"name":"value","type":"uint256"},{"indexed":true,"name":"fromAddress","type":"address"}],"name":"PunkBidWithdrawn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"punkIndex","type":"uint256"},{"indexed":false,"name":"value","type":"uint256"},{"indexed":true,"name":"fromAddress","type":"address"},{"indexed":true,"name":"toAddress","type":"address"}],"name":"PunkBought","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"punkIndex","type":"uint256"}],"name":"PunkNoLongerForSale","type":"event"}]"""
punk_contract = w3.eth.contract(address="0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB", abi = abi_text)
event_filter = punk_contract.events.PunkBought.createFilter(fromBlock = hexDaysAgo(w3, QUERY_DAYS))
punk_events = event_filter.get_all_entries()
len(punk_events)

2227

In [522]:
ids = []
values = []
for e in punk_events:
    if (e.args.value != 0): #not sure why
        ids.append(e.args.punkIndex)
        values.append(e.args.value / 1e18) # PROBABLY UNSAFE
    
df_sell = pd.DataFrame(values, index=ids)
df_sell.columns = ['price']
df_sell.index.name = 'id'

b = pd.merge(df, df_sell, how='inner', on='id')

In [523]:
b = b.drop(columns=['Male', 'Female'])

In [524]:
perc_train = .7
split_num = round(len(x) * perc_train)

x = b.loc[:, b.columns != 'price'].to_numpy()
y = b['price'].to_numpy()

x_train = x[0:split_num]
x_test = x[split_num:]

y_train = y[0:split_num]
y_test = y[split_num:]


In [525]:
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model = model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('Coefficients: \n', model.coef_)
print('Mean squared error: %.2f'% mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f'% r2_score(y_test, y_pred))

Coefficients: 
 [ 5.12119793e+12  4.18160547e+03  6.37558594e+02  2.31898438e+02
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
  5.31111547e+02 -5.12119793e+12 -5.12119793e+12 -5.12119793e+12
 -5.12119

In [520]:
b.loc[df['Zombie'] == 1]

,attr_count,Alien,Ape,Zombie,price
id,,,,,
1119,3,0,0,1,300.0
2484,2,0,0,1,175.0
2681,2,0,0,1,277.0
3609,2,0,0,1,369.0
